In [1]:
from bokeh.plotting import figure, show, output_notebook
import pandas as pd
import numpy as np
import os

### Load Data & Feature Selection & Transform

In [2]:
## load_raw_data
DISEASE_NAME = 'HS_iCDf'
DATASET_FOLDER = os.path.join('..', 'datasets/ibd_dataset', DISEASE_NAME)
training_file = os.path.join(DATASET_FOLDER, 'Sokol_16S_taxa_HS_iCDf_commsamp_training.txt')
training_labs = os.path.join(DATASET_FOLDER, 'Sokol_16S_taxa_HS_iCDf_commsamp_training_lab.txt')
test_file = os.path.join(DATASET_FOLDER, 'Sokol_16S_taxa_HS_iCDf_commsamp_test.txt')
test_labs = os.path.join(DATASET_FOLDER, 'Sokol_16S_taxa_HS_iCDf_commsamp_test_lab.txt')
df_train = pd.read_csv(training_file,  sep='\t', index_col=0, header=0)
df_test = pd.read_csv(test_file,  sep='\t', index_col=0, header=0)
y_train, y_test = np.loadtxt(training_labs, dtype=np.int), np.loadtxt(test_labs, dtype=np.int)

In [3]:
from sklearn.feature_selection import SelectKBest
kbest = SelectKBest(k=62)
kbest = kbest.fit(df_train, y_train)
index_ranked = kbest.get_support(True)
cols = list()
for i in index_ranked:
    cols.append(df_train.columns.values[i])
df_train_fs = df_train[cols]
df_test_fs = df_test[cols] 

D:\devtools\anaconda\envs\xbiome\lib\site-packages\sklearn\feature_selection\_univariate_selection.py:115: UserWarning: Features [  6  47  65  76 112 187 235] are constant.
  UserWarning)
D:\devtools\anaconda\envs\xbiome\lib\site-packages\sklearn\feature_selection\_univariate_selection.py:116: RuntimeWarning: invalid value encountered in true_divide
  f = msb / msw


In [4]:
X_train = df_train_fs.astype(dtype=np.float32)
X_test = df_test_fs.astype(dtype=np.float32)
X = np.vstack((X_train, X_test))
y = np.hstack((y_train, y_test))

In [5]:
X_train.shape, X_test.shape,X.shape, y.shape

((58, 62), (24, 62), (82, 62), (82,))

In [6]:
## Load Coordinates
COORD_FILEPATH = os.path.join('..', 'datasets/coordinates/', 'coordinates_icdf.txt')
coords = pd.read_csv(COORD_FILEPATH, sep='\t', 
                     header=0, index_col=0)
coords = coords.astype(dtype=np.float32)
nb_coordinates = coords.shape[0]

In [7]:
coords.shape

(306, 247)

### Visualise Embeddings of Data (ICDF)

Color Code: 

* <span style="color: green;"> Healty Subjects</span>
* <span style="color: red;"> ICDf Subjects</span>


In [8]:
colormap = {1: 'green', 0: 'red'}
colors = [colormap[v] for v in y]
from sklearn.manifold._t_sne import TSNE
tsne = TSNE(n_components=2, n_iter=5000, perplexity=5)
X_tsne = tsne.fit_transform(X)
X.shape, X_tsne.shape
p = figure(title = "iCDf Data", plot_width=400, plot_height=400)
p.xaxis.axis_label = 'First Component'
p.yaxis.axis_label = 'Second Component'

p.circle(X_tsne[:, 0], X_tsne[:,1],
         color=colors, fill_alpha=0.2, size=10)
show(p)

### Extract PhyloConv Layer Activations

In [9]:
from keras.backend import floatx
from keras.engine import Input, Model
from keras.layers import (Lambda, MaxPooling1D, Flatten,
                          Dropout, Dense, BatchNormalization)


Using TensorFlow backend.
D:\devtools\anaconda\envs\xbiome\lib\site-packages\tensorflow\python\framework\dtypes.py:523: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
D:\devtools\anaconda\envs\xbiome\lib\site-packages\tensorflow\python\framework\dtypes.py:524: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
D:\devtools\anaconda\envs\xbiome\lib\site-packages\tensorflow\python\framework\dtypes.py:525: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
D:\devtools\anaconda\envs\xbiome\lib\site-packages\tenso

In [10]:
import os
import sys

# Change PYTHONPATH to allow for relative import
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
from phcnn.layers import PhyloConv1D, euclidean_distances

WARNING (theano.configdefaults): g++ not available, if using conda: `conda install m2w64-toolchain`
D:\devtools\anaconda\envs\xbiome\lib\site-packages\theano\configdefaults.py:560: UserWarning: DeprecationWarning: there is no c++ compiler.This is deprecated and with Theano 0.11 a c++ compiler will be mandatory
  warnings.warn("DeprecationWarning: there is no c++ compiler."
WARNING (theano.configdefaults): g++ not detected ! Theano will be unable to execute optimized C-implementations (for both CPU and GPU) and will default to Python implementations. Performance will be severely degraded. To remove this warning, set Theano flags cxx to an empty string.
WARNING (theano.tensor.blas): Using NumPy C-API based implementation for BLAS functions.


In [11]:
all_coordinates = np.empty((X.shape[0],) + coords.shape, dtype=np.float64)
for i in range(X.shape[0]):
    all_coordinates[i] = coords

In [12]:
X.shape,coords.shape,all_coordinates.shape

((82, 62), (306, 247), (82, 306, 247))

In [13]:
all_coordinates = all_coordinates[..., np.newaxis]
X = X[..., np.newaxis]

In [14]:
all_coordinates.shape, X.shape

((82, 306, 247, 1), (82, 62, 1))

In [157]:
nb_features = 62  # current nb of features in the feature step!
# Paramenters for phylo_conv layers
list_filters = [16, 16]
list_neighbours = [4, 4]
# Parameter for output layer
nb_classes = 2

In [158]:
from keras import backend as K

In [159]:
WEIGHT_FILE = os.path.join(os.path.abspath(os.path.curdir), 'icdf_phcnn_model_weights.hdf5')

#### Create PhCNN Network Model

In [160]:
K.clear_session()
data = Input(shape=(nb_features, 1), name="data", dtype=floatx())
coordinates = Input(shape=(nb_coordinates, nb_features, 1),
                    name="coordinates", dtype=floatx())


conv_layer = data
# We remove the padding that we added to work around keras limitations
conv_crd = Lambda(lambda c: c[0], output_shape=lambda s: (s[1:]))(coordinates)

In [161]:
for nb_filters, nb_neighbors in zip(list_filters, list_neighbours):

    if nb_neighbors > nb_features:
        raise Exception("More neighbors than features, "
                        "please use less neighbors or use more features")

    distances = euclidean_distances(conv_crd)
    conv_layer, conv_crd = PhyloConv1D(distances,  
                                       nb_neighbors, 
                                       nb_filters,activation='selu', trainable=False)([conv_layer, conv_crd])
    
# import pdb;pdb.set_trace()
maxp = MaxPooling1D(pool_size=2, padding="valid")(conv_layer)
flatt = Flatten()(maxp)
fc_1 = Dense(units= 128, name='fc_1', activation='sigmoid')(flatt)
dropout = Dropout(0.25, name="dropout_l")(fc_1)
output = Dense(units=nb_classes, kernel_initializer="he_normal",
               activation="softmax", name='new_output')(dropout)

model = Model(inputs=[data, coordinates], outputs=output)

#model.load_weights(WEIGHT_FILE, by_name=True)
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
coordinates (InputLayer)        (None, 306, 62, 1)   0                                            
__________________________________________________________________________________________________
data (InputLayer)               (None, 62, 1)        0                                            
__________________________________________________________________________________________________
lambda_1 (Lambda)               (306, 62, 1)         0           coordinates[0][0]                
__________________________________________________________________________________________________
phylo_conv1d_1 (PhyloConv1D)    [(None, 62, 16), (30 80          data[0][0]                       
                                                                 lambda_1[0][0]                   
__________

### Conv Filters

In [163]:
def get_conv_activations(model, layer, Coords, X_batch):
    """Tensorflow function to extract activations from target layer tensor(s).
    This function is supposed to be applied to a Convolutional Layer, so the
    outputs of the function will be the list of all tensors (i.e. cnn filters.)
    
    Parameters:
    -----------
    model: keras.models.Model
        Instance of the target model (i.e. PhCNN network)
    layer: keras.layers.convolutional._Conv
        Instance of the target Convolutional Layer from which 
        extract output tensors
    Coords:  array-like, shape: (batch_size, nb_coords, nb_features, 1)
        Tensor of Coordinates
    X_batch: array-like, shape: (batch_size, nb_features, 1)
        Tensor of batch data
    
    Returns:
    --------
        List of tensor objects.
    """
    
    activations_f = K.function(inputs=[model.layers[0].input, 
                                       model.layers[1].input,
                                       K.learning_phase()], 
                               outputs=[t for t in layer.output])
    
    activations = activations_f((Coords, X_batch, False))
    return activations

#### Specify Target Network Layer

In [169]:
phyloconv_layer_1 = model.layers[3]

In [170]:
phyloconv_layer_1.name

'phylo_conv1d_1'

In [171]:
model.layers[0].input, model.layers[1].input,K.learning_phase()

(<tf.Tensor 'coordinates:0' shape=(?, 306, 62, 1) dtype=float32>,
 <tf.Tensor 'data:0' shape=(?, 62, 1) dtype=float32>,
 <tf.Tensor 'keras_learning_phase:0' shape=() dtype=bool>)

In [172]:
phyloconv_layer_1.output

[<tf.Tensor 'phylo_conv1d_1/mul_1:0' shape=(?, 62, 16) dtype=float32>,
 <tf.Tensor 'phylo_conv1d_1/mul_3:0' shape=(306, 62, 16) dtype=float32>]

#### Get PhyloConv Activations

In [173]:
conv_acts = get_conv_activations(model, phyloconv_layer_1, all_coordinates, X)

InvalidArgumentError: transpose expects a vector of size 7. But input(1) is a vector of size 3
	 [[{{node phylo_conv1d_1/transpose_2}} = Transpose[T=DT_FLOAT, Tperm=DT_INT32, _device="/job:localhost/replica:0/task:0/device:CPU:0"](_arg_data_0_1, phylo_conv1d_1/transpose_2/perm)]]

In [139]:
conv_acts[0].shape

NameError: name 'conv_acts' is not defined

In [46]:
X_cnn = conv_acts[0]

NameError: name 'conv_acts' is not defined

#### Plot Embeddings of PhCnn Layer

In [45]:
tsne = TSNE(n_components=2, n_iter=5000, perplexity=5)
for i in range(16):  # nb filters
    try:
        X_cnn_tsne = tsne.fit_transform(X_cnn[:, :, i])
        p = figure(title = "PhyloConv filter {}".format(i+1), 
                   plot_width=400, plot_height=400)
        p.xaxis.axis_label = 'First Component'
        p.yaxis.axis_label = 'Second Component'

        p.circle(X_cnn_tsne[:, 0], X_cnn_tsne[:,1],
                 color=colors, fill_alpha=0.2, size=10)
        show(p)
    except ValueError:
        continue

NameError: name 'X_cnn' is not defined